### Dependencies

In [1]:
!git clone "https://github.com/suyash/ContextualDecomposition.git" && mv ContextualDecomposition/cd ./cd

Cloning into 'ContextualDecomposition'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 35 (delta 13), reused 33 (delta 11), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [2]:
!curl -L -o "job_dir.zip" "https://drive.google.com/uc?export=download&id=13Uyub6pPWWS9USmj2WxAilPMoYFQZtCd" && unzip -q -d "job_dir" "job_dir.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0     35      0 --:--:--  0:00:10 --:--:--    81
100 21.6M    0 21.6M    0     0  1899k      0 --:--:--  0:00:11 --:--:-- 60.9M


### Imports

In [3]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model  # pylint: disable=import-error
from tensorflow.keras.layers import Dense, Dropout, Embedding, Input, LSTM  # pylint: disable=import-error
import tensorflow_datasets as tfds

from cd.cd import lstm_decomposition
from cd.lstm import create_table, create_inv_table, preprocess_dataset

In [5]:
model = tf.keras.models.load_model("job_dir/saved_model/best")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         1831808   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 1,963,650
Trainable params: 1,963,650
Non-trainable params: 0
_________________________________________________________________


In [0]:
tokens = np.load("job_dir/tokens.npy")
table = create_table(tokens)
inv_table = create_inv_table(tokens)

### Process Input

In [0]:
s = "it 's easy to love robin tunney -- she is pretty and she can act -- but it gets harder and harder to understand her choices ."

In [8]:
inp = table.lookup(tf.constant(s.split()))
inp = tf.expand_dims(inp, 0)
inp

<tf.Tensor: shape=(1, 27), dtype=int32, numpy=
array([[  12,    8,  396,    7,   82, 3814, 5186,   28,  281,    9,  313,
           4,  281,   66,  590,   28,   22,   12,  283, 5449,    4, 5449,
           7,  683,  115, 2049,    6]], dtype=int32)>

### Generate Overall Prediction

In [9]:
x = model.predict(inp)
x = tf.math.softmax(x)
x

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.72758025, 0.27241975]], dtype=float32)>

$P(neg) = 0.84$, $P(pos) = 0.15$

Now, decomposing and getting predictions for subsections

In [10]:
t = inv_table.lookup(inp[0]).numpy()
list(enumerate(t))

[(0, b'it'),
 (1, b"'s"),
 (2, b'easy'),
 (3, b'to'),
 (4, b'love'),
 (5, b'robin'),
 (6, b'tunney'),
 (7, b'--'),
 (8, b'she'),
 (9, b'is'),
 (10, b'pretty'),
 (11, b'and'),
 (12, b'she'),
 (13, b'can'),
 (14, b'act'),
 (15, b'--'),
 (16, b'but'),
 (17, b'it'),
 (18, b'gets'),
 (19, b'harder'),
 (20, b'and'),
 (21, b'harder'),
 (22, b'to'),
 (23, b'understand'),
 (24, b'her'),
 (25, b'choices'),
 (26, b'.')]

#### Prepare the Embedding generator for the input

In [0]:
def prepare_embedder(embed_dim, vocab_size):
    seq = Input((None, ), dtype="int32")
    x = Embedding(
        vocab_size,
        embed_dim,
    )(seq)
    return tf.keras.Model(seq, x)

In [0]:
embedder = prepare_embedder(128, 2 + len(tokens))

In [0]:
embedder.set_weights(model.get_weights()[:1])

### Decomposing the prediction into the prediction for [0..15] and [16..26]

In [0]:
_, k, rk, b, dw, db = model.weights

In [0]:
embed_inp = embedder(inp)

In [0]:
pred_0_15, _ = lstm_decomposition(embed_inp, k, rk, b, 0, 15)
pred_16_26, _ = lstm_decomposition(embed_inp, k, rk, b, 16, 26)

In [17]:
tf.math.softmax(tf.matmul(pred_0_15, dw) + db)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.00325666, 0.9967434 ]], dtype=float32)>

In [18]:
tf.math.softmax(tf.matmul(pred_16_26, dw) + db)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.9955011 , 0.00449888]], dtype=float32)>

decomposed prediction for __"it 's easy to love robin tunney - she is pretty and she can act -"__: $P(neg) = 0.005, P(pos) = 0.995$

decomposed prediction for __"but it gets harder and harder to understand her choices ."__: $P(neg) = 0.996, P(pos) = 0.004$

### Individual Word Level Decomposition

In [0]:
preds = []
for i in range(27):
    rel, _ = lstm_decomposition(embed_inp, k, rk, b, i, i)
    pred = tf.math.softmax(tf.matmul(rel, dw) + db)
    preds.append(pred.numpy().tolist()[0])

In [20]:
list(zip(t, preds))

[(b'it', [0.4859467148780823, 0.5140532851219177]),
 (b"'s", [0.5432335138320923, 0.4567664563655853]),
 (b'easy', [0.4714977443218231, 0.5285022854804993]),
 (b'to', [0.5675297975540161, 0.4324701726436615]),
 (b'love', [0.14848144352436066, 0.8515185117721558]),
 (b'robin', [0.3177938759326935, 0.6822061538696289]),
 (b'tunney', [0.4475875198841095, 0.5524125099182129]),
 (b'--', [0.5062448978424072, 0.4937551021575928]),
 (b'she', [0.32740598917007446, 0.6725940108299255]),
 (b'is', [0.4344940185546875, 0.5655059814453125]),
 (b'pretty', [0.22031214833259583, 0.7796878218650818]),
 (b'and', [0.43068253993988037, 0.5693174600601196]),
 (b'she', [0.35035571455955505, 0.6496442556381226]),
 (b'can', [0.4265875518321991, 0.5734124779701233]),
 (b'act', [0.31614911556243896, 0.6838509440422058]),
 (b'--', [0.4745381474494934, 0.5254618525505066]),
 (b'but', [0.32750606536865234, 0.6724939346313477]),
 (b'it', [0.5035586357116699, 0.4964413642883301]),
 (b'gets', [0.546697199344635, 0.453